# ストーリー
- 再学習時にドリフトと、精度異常が起きている
- ドリフトは分析した以降の日付に起こるようにする（特定のパラメータでだけ生産を行った）
- 精度異常はコンセプトドリフトが起きていて、塗布液自体に変化があった。
- 塗布液の情報を加えた再学習モデルと、加えてない再学習モデルを準備して、チャレンジャーで比較する

In [5]:
! pip install -r requirements.txt

  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.0/615.0 kB 222.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 189.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 385.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 235.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 240.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 365.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 221.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 383.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 173.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 372.5 MB/s eta 0:00:00
  Created wheel for pulumi-datarobot: filename=pulumi_datarobot-0.

In [2]:
from datetime import date, timedelta
import random

import pandas as pd
import numpy as np

import datarobot as dr


df = pd.read_csv("../assets/コーティング製品ブリードアウトmain_train.csv")
# 塗布長を短いやつだけにする
sample = df.sample(n=200, random_state=42)


def generate_binary_list(n, p):
  """
  0と1の整数のリストを生成する。

  Args:
    n: リストの長さ。
    p: 1が出現する確率。

  Returns:
    0と1の整数のリスト。

  # パラメータ設定
  n = 200  # リストの長さ
  ratio_0 = 3  # 0の比率
  ratio_1 = 1  # 1の比率
  p = ratio_1 / (ratio_0 + ratio_1)  # 1が出現する確率

  # リスト生成
  binary_list = generate_binary_list(n, p)
  """
  return np.random.binomial(1, p, n).tolist()

# データドリフト用のデータを作る

# パラメータ設定
n = 200  # リストの長さ
ratio_0 = 3  # 0の比率
ratio_1 = 1  # 1の比率
p = ratio_1 / (ratio_0 + ratio_1)  # 1が出現する確率

# リスト生成
binary_list = generate_binary_list(n, p)

sample["塗布長"] = binary_list
sample["塗布長"] = sample["塗布長"].map({0:"30m", 1:"100m"})



# 予測時に日付の情報を乗せたいので日付を作る
def get_last_week_dates():
  """今日から1週間前までの毎日の日付を"%Y-%m-%d"形式のリストで返す。"""
  today = date.today()
  dates = []
  for i in range(7):
    current_date = today - timedelta(days=i)
    dates.append(current_date.strftime("%Y-%m-%d"))
  return dates

# 実行例
last_week_dates = get_last_week_dates()
print(last_week_dates)

# データドリフトが起きてないデータをサンプルして作る。
sample_list = []
for i in range(6):
    _df =  df.sample(n=200, random_state=i)
    _df["date_col"] = last_week_dates.pop(0)
    sample_list.append(_df)
sample["date_col"] = last_week_dates.pop(0)
sample_list.append(sample)

# ランダムに割り付けたdataframeを一つにする
sample = pd.concat(sample_list)

# 精度の監視のためにロット番号を更新する
lot_num = []
init_num = 10000
for i in range(sample.shape[0]):
    lot_num.append(f"SC{str(init_num).zfill(7)}")
    init_num += 1
sample["ロット番号"] = lot_num

# 保存する
sample.to_csv("data_drift.csv", index=False)

# 翌週のデータを作る
def get_next_week_dates():
  """今日から1週間前までの毎日の日付を"%Y-%m-%d"形式のリストで返す。"""
  today = date.today()
  dates = []
  for i in range(1, 8):
    current_date = today + timedelta(days=i)
    dates.append(current_date.strftime("%Y-%m-%d"))
  return dates

# 実行例
next_week_dates = get_next_week_dates()
print(next_week_dates)

# サンプリングして日付をつける
sample_list = []
for i in range(8, 15):
    sample = df.sample(n=200, random_state=i)
    sample["date_col"] = next_week_dates.pop(0)
    sample_list.append(sample)
sample_df_next_week = pd.concat(sample_list)

# ロット番号をつける
lot_num = []
init_num = 11400
for i in range(sample_df_next_week.shape[0]):
    lot_num.append(f"SC{str(init_num).zfill(7)}")
    init_num += 1
sample_df_next_week["ロット番号"] = lot_num
# 保存する
sample_df_next_week.to_csv("next_week.csv", index=False)


# 精度のためにデータを確率で入れ替える
def replace_false_with_true_numpy(binary_list, probability):
  """
  NumPyを使ってFalseをTrueに置き換える関数。

  Args:
    binary_list: TrueとFalseのバイナリリスト。
    probability: FalseをTrueに置き換える確率 (0から1の範囲)。

  Returns:
    FalseがTrueに置き換えられた新しいバイナリリスト。
  """
  arr = np.array(binary_list)
  random_values = np.random.rand(len(arr))
  arr = np.logical_or(arr, random_values < probability)
  return arr.tolist()

sample_last_week = pd.read_csv("data_drift.csv")
bleedout = []

# 少しづつ異常が増えていくようにする
for date in sample_last_week["date_col"].unique():
    _tmp = sample_last_week[sample_last_week["date_col"] == date]
    binary_tmp = _tmp["ブリードアウト"].tolist()
    if date == '2025-03-14':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.02)
    if date == '2025-03-15':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.04)
    if date == '2025-03-16':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.06)
    bleedout.extend(binary_tmp)

sample_last_week["ブリードアウト"] = bleedout

# 保存する
sample_last_week.to_csv("data_drift_modify.csv", index=False)
sample_last_week[["ロット番号", "ブリードアウト"]].to_csv("正解データ1.csv",index=False)

# 先の日付についても同様に行う
bleedout = []

for date in sample_df_next_week["date_col"].unique():
    _tmp = sample_df_next_week[sample_df_next_week["date_col"] == date]
    binary_tmp = _tmp["ブリードアウト"].tolist()
    if date == '2025-03-17':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.08)
    if date == '2025-03-18':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.1)
    if date == '2025-03-19':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.12)
    if date == '2025-03-20':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.14)
    if date == '2025-03-21':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.16)
    if date == '2025-03-22':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.18)
    if date == '2025-03-23':
        binary_tmp = replace_false_with_true_numpy(binary_tmp, 0.2)
    bleedout.extend(binary_tmp)


sample_df_next_week["ブリードアウト"] = bleedout
sample_df_next_week.to_csv("next_week_modify.csv", index=False)
sample_df_next_week[["ロット番号", "ブリードアウト"]].to_csv("正解データ2.csv",index=False)

['2025-03-19', '2025-03-18', '2025-03-17', '2025-03-16', '2025-03-15', '2025-03-14', '2025-03-13']
['2025-03-20', '2025-03-21', '2025-03-22', '2025-03-23', '2025-03-24', '2025-03-25', '2025-03-26']


# 新しい行の追加と予測用にデータ保存

In [1]:
rate = 3
import pandas as pd

# 過去データ読み込み
original_data = pd.read_csv('../data_drift_original.csv')
drift_data = pd.read_csv("../data_drift_modify.csv")
new_data = drift_data.copy()
new_data["塗布材料"] = "A"

# 塗布材料作成
new_data.loc[original_data["ブリードアウト"] != drift_data["ブリードアウト"], "塗布材料"] = "B"

# 新しい塗布材料のインデックス指定
replace_index = []
for i in new_data["date_col"].unique():
    if i == '2025-03-14':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.02/rate), random_state=42).index)
        #replace_index.append()
    if i == '2025-03-15':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.04/rate), random_state=42).index)
    if i == '2025-03-16':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.06/rate), random_state=42).index)

# 置換
new_data.loc[replace_index, "塗布材料"] = "B"

# 日付順にソート
new_data["date_col"] = pd.to_datetime(new_data["date_col"])
new_data = new_data.sort_values(by='date_col', ascending=True)
lot_num = []
init_num = 10000
for i in range(drift_data.shape[0]):
    lot_num.append(f"SC{str(init_num).zfill(7)}")
    init_num += 1
new_data['ロット番号'] = lot_num
new_data = new_data.reset_index(drop=True)

# 書き出し
new_data.to_csv('../assets/data_01.csv', index=False)

In [2]:
original_data = pd.read_csv('../next_week.csv')
drift_data = pd.read_csv("../next_week_modify.csv")
new_data = drift_data.copy()
new_data["塗布材料"] = "A"

# 塗布材料の行作成

new_data.loc[original_data["ブリードアウト"] != drift_data["ブリードアウト"], "塗布材料"] = "B"

# 新しい塗布材料のインデックス指定

replace_index = []
for i in new_data["date_col"].unique():
    if i == '2025-03-17':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.08/rate), random_state=42).index)
        #replace_index.append()
    if i == '2025-03-18':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.1/rate), random_state=42).index)
    if i == '2025-03-19':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.12/rate), random_state=42).index)
        #replace_index.append()
    if i == '2025-03-20':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.14/rate), random_state=42).index)
    if i == '2025-03-21':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.16/rate), random_state=42).index)
        #replace_index.append()
    if i == '2025-03-22':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.18/rate), random_state=42).index)

    if i == '2025-03-23':
        _tmp = new_data[(new_data["date_col"]==i) & (new_data["ブリードアウト"]==False)]
        replace_index.extend(_tmp.sample(n=int(_tmp.shape[0]*0.2/rate), random_state=42).index)


# 置換
new_data.loc[replace_index, "塗布材料"] = "B"

new_data.to_csv('../assets/data_02.csv', index=False)

In [3]:
data_01 = pd.read_csv("../assets/data_01.csv")
data_02 = pd.read_csv("../assets/data_02.csv")
df = pd.read_csv("../assets/コーティング製品ブリードアウトmain_train.csv")

# 02（将来のデータ）と塗布液変更データを含まずに再学習用のデータとした
pd.concat([df, data_01.drop(columns=['date_col', '塗布材料'])]).to_csv('../assets/再学習用データ01.csv', index=False)

# 02（将来のデータ）なしで塗布液変更データを含んで再学習用のデータとした
df['塗布材料'] = "A"
pd.concat([df, data_01.drop(columns=['date_col'])]).to_csv('../assets/再学習用データ02.csv', index=False)

# 02（将来のデータ）を含んで再学習用のデータとした。
pd.concat([df, data_01.drop(columns=['date_col']), data_02.drop(columns=['date_col'])]).to_csv('../assets/再学習用データ03.csv', index=False)

# todo
# 将来データ移行の予測データも作る。

# 予測用データと実績値データを作る。
prediction_data = pd.concat([data_01, data_02])
prediction_data.drop(columns=['ブリードアウト', '塗布材料']).to_csv("../assets/prediction_data.csv", index=False)
prediction_data[['ロット番号', 'ブリードアウト']].to_csv("../assets/prediction_data_actual.csv", index=False)